# 다중 벡터 하이브리드 검색
- 텍스트와 이미지가 포홤된 트윗의 경우 텍스트나 이미지 중 하나가 검색 쿼리의 의미와 일치하면 검색됨
- 하이브리드 검색은 다양한 분야의 검색을 결합하여 검색 경험을 향상시킴
- 밀버스는 여러 벡터 필드에 대한 검색을 허용하고 여러 개의 근사 이웃(ANN) 검색을 동시에 수행함으로써 이를 지원

참고자료: https://milvus.io/docs/ko/multi-vector-search.md

## 다양한 임베딩 방식
- 스파스-밀도 벡터 검색: 고밀도 벡터는 의미론적 관계를 포착하는 데 탁월한 반면, 스파스 벡터는 정확한 키워드 매칭에 매우 효과적, 하이브리드 검색은 이러한 접근 방식을 결합하여 폭넓은 개념 이해와 정확한 용어 관련성을 모두 제공함으로써 검색 결과를 개선
- 멀티모달 벡터 검색: 멀티 모달 벡터 검색은 텍스트, 이미지, 오디오 등 다양한 데이터 유형에 걸쳐 검색할 수 있는 기술, 이 접근 방식의 가장 큰 장점은 다양한 방식을 매끄럽고 일관된 검색 환경으로 통합할 수 있다는 점, 예를 들어, 제품 검색에서 사용자가 텍스트 쿼리를 입력하면 텍스트와 이미지로 설명된 제품을 모두 찾을 수 있음

## 예시
- 각 제품에 텍스트 설명과 이미지가 포함되어 있는 실제 사용 사례를 고려
- 사용 가능한 데이터를 기반으로 세 가지 유형의 검색을 수행

- 시맨틱 텍스트 검색: 여기에는 고밀도 벡터를 사용하여 제품의 텍스트 설명을 쿼리하는 것이 포함됨, 텍스트 임베딩은 BERT, Transformers와 같은 모델이나 OpenAI와 같은 서비스 사용하여 만듦
- 전체 텍스트 검색: Sparse vector와 키워드 일치를 사용하여 제품의 텍스트 설명을 쿼리함, 이를 위해 BM25, BGE-M3, SPLADE와 같은 Sparse embedding 모델을 활용할 수 있음
- 멀티모달 이미지 검색: 고밀도 벡터가 포함된 텍스트 쿼리를 사용하여 이미지를 쿼리함, 이미지 임베딩은 CLIP과 같은 모델을 사용하여 생성함

## 여러 벡터 필드가 있는 컬렉션 만들기
- 컬렉션을 만드는 과정에는 컬렉션 스키마 정의, 인덱스 매개변수 구성, 컬렉션 생성의 세 가지 주요 단계가 포함됨

### 스키마 정의
- 다중 벡터 하이브리드 검색의 경우, 컬렉션 스키마 내에 여러 개의 벡터 필드를 정의
- 기본적으로 각 컬렉션은 최대 4개의 벡터 필드를 수용할 수 있음
- 필요에 따라 컬렉션에 최대 10개의 벡터 필드를 포함하도록 proxy.maxVectorFieldNum을 조정할 수 있음


In [1]:
from pymilvus import (
    MilvusClient, DataType, Function, FunctionType
)

client = MilvusClient(
    uri="http://localhost:19530",
    token="root:Milvus"
)

schema = MilvusClient.create_schema(auto_id=False)

schema.add_field(field_name="id", datatype=DataType.INT64, is_primary=True, description="product id")
schema.add_field(field_name="text", datatype=DataType.VARCHAR, max_length=1000, enable_analyzer=True, description="raw text of product description")
schema.add_field(field_name="text_dense", datatype=DataType.FLOAT_VECTOR, dim=768, description="text dense embedding")
schema.add_field(field_name="text_sparse", datatype=DataType.SPARSE_FLOAT_VECTOR, description="text sparse embedding auto-generated by the built-in BM25 function")
schema.add_field(field_name="image_dense", datatype=DataType.FLOAT_VECTOR, dim=512, description="image dense embedding")

bm25_function = Function(
    name="text_bm25_emb",
    input_field_names=["text"],
    output_field_names=["text_sparse"],
    function_type=FunctionType.BM25,
)

schema.add_function(bm25_function)

{'auto_id': False, 'description': '', 'fields': [{'name': 'id', 'description': 'product id', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'text', 'description': 'raw text of product description', 'type': <DataType.VARCHAR: 21>, 'params': {'max_length': 1000, 'enable_analyzer': True}}, {'name': 'text_dense', 'description': 'text dense embedding', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 768}}, {'name': 'text_sparse', 'description': 'text sparse embedding auto-generated by the built-in BM25 function', 'type': <DataType.SPARSE_FLOAT_VECTOR: 104>, 'is_function_output': True}, {'name': 'image_dense', 'description': 'image dense embedding', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 512}}], 'enable_dynamic_field': False, 'functions': [{'name': 'text_bm25_emb', 'description': '', 'type': <FunctionType.BM25: 1>, 'input_field_names': ['text'], 'output_field_names': ['text_sparse'], 'params': {}}]}

In [2]:
from pymilvus import MilvusClient

index_params = client.prepare_index_params()

index_params.add_index(
    field_name="text_dense",
    index_name="text_dense_index",
    index_type="AUTOINDEX",
    metric_type="IP"
)

index_params.add_index(
    field_name="text_sparse",
    index_name="text_sparse_index",
    index_type="SPARSE_INVERTED_INDEX",
    metric_type="BM25",
    params={"inverted_index_algo": "DAAT_MAXSCORE"}, # or "DAAT_WAND" or "TAAT_NAIVE"
)

index_params.add_index(
    field_name="image_dense",
    index_name="image_dense_index",
    index_type="AUTOINDEX",
    metric_type="IP"
)


In [4]:
from pymilvus import MilvusClient

client.create_collection(
    collection_name="my_collection_multi",
    schema=schema,
    index_params=index_params
)

# 데이터 삽입
- 이 섹션에서는 앞서 정의한 스키마에 따라 my_collection_multi 컬렉션에 데이터를 삽입

In [ ]:
from pymilvus import MilvusClient

data=[
    {
        "id": 0,
        "text": "Red cotton t-shirt with round neck",
        "text_dense": [0.3580376395471989, -0.6023495712049978, 0.18414012509913835, ...],
        "image_dense": [0.6366019600530924, -0.09323198122475052, ...]
    },
    {
        "id": 1,
        "text": "Wireless noise-cancelling over-ear headphones",
        "text_dense": [0.19886812562848388, 0.06023560599112088, 0.6976963061752597, ...],
        "image_dense": [0.6414180010301553, 0.8976979978567611, ...]
    },
    {
        "id": 2,
        "text": "Stainless steel water bottle, 500ml",
        "dense": [0.43742130801983836, -0.5597502546264526, 0.6457887650909682, ...],
        "image_dense": [-0.6901259768402174, 0.6100500332193755, ...]
    }
]

res = client.insert(
    collection_name="my_collection",
    data=data
)



## 하이브리드 검색 수행
### 여러 개의 AnnSearchRequest 인스턴스 생성하기
- 하이브리드 검색은 hybrid_search() 함수에 AnnSearchRequest을 여러 개 생성하여 구현되며, 각 AnnSearchRequest은 특정 벡터 필드에 대한 기본 ANN 검색 요청을 나타냄
- 또한 AnnSearchRequest에서 expr 파라미터를 구성하여 하이브리드 검색의 필터링 조건을 설정할 수 있음

In [ ]:
from pymilvus import AnnSearchRequest

query_text = "white headphones, quiet and comfortable"
query_dense_vector = [0.3580376395471989, -0.6023495712049978, 0.5142999509918703, ...]
query_multimodal_vector = [0.015829865178701663, 0.5264158340734488, ...]

search_param_1 = {
    "data": [query_dense_vector],
    "anns_field": "text_dense",
    "param": {"nprobe": 10},
    "limit": 2
}
request_1 = AnnSearchRequest(**search_param_1)

search_param_2 = {
    "data": [query_text],
    "anns_field": "text_sparse",
    "param": {"drop_ratio_search": 0.2},
    "limit": 2
}
request_2 = AnnSearchRequest(**search_param_2)

search_param_3 = {
    "data": [query_multimodal_vector],
    "anns_field": "image_dense",
    "param": {"nprobe": 10},
    "limit": 2
}
request_3 = AnnSearchRequest(**search_param_3)

reqs = [request_1, request_2, request_3]

### 순위 재조정 전략 구성
- ANN 검색 결과 집합을 병합하고 re-rank하려면 적절한 re-rank 전략을 선택해야함
- 두 가지 유형의 re-rank 전략을 제공
    - 가중 순위: 결과에서 특정 벡터 필드를 강조해야 하는 경우 이 전략을 사용, 가중랭커를 사용하면 특정 벡터 필드에 더 큰 가중치를 할당하여 더 눈에 띄게 강조할 수 있음
    - RRFRanker(상호 순위 융합 랭커): 특별히 강조할 필요가 없는 경우 사용, 각 벡터 필드의 중요도를 효과적으로 균형있게 조정

In [7]:
from pymilvus import RRFRanker

ranker = RRFRanker(100)

## 하이브리드 검색 수행
- 하이브리드 검색을 시작하기 전에 컬렉션이 로드되었는지 확인

In [ ]:
from pymilvus import MilvusClient

res = client.hybrid_search(
    collection_name="my_collection",
    reqs=reqs,
    ranker=ranker,
    limit=2
)
for hits in res:
    print("TopK results:")
    for hit in hits:
        print(hit)
